In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('data/train.csv')

In [4]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# Convert "null" values to None
train[["LotFrontage",'MasVnrArea','GarageYrBlt']]=train[["LotFrontage",'MasVnrArea','GarageYrBlt']].replace(np.nan,0)
train[['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature','MasVnrType','Electrical']]=train[['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature','MasVnrType','Electrical']].replace(np.nan,"no value")

In [6]:
inputs = train.drop(columns=['SalePrice', 'Id'])
# inputs = inputs[['LotArea', 'MSSubClass']]
inputs = pd.get_dummies(inputs)
inputs.head()
inputs = inputs.values
inputs

array([[6.000e+01, 6.500e+01, 8.450e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+01, 8.000e+01, 9.600e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [6.000e+01, 6.800e+01, 1.125e+04, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [7.000e+01, 6.600e+01, 9.042e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+01, 6.800e+01, 9.717e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [2.000e+01, 7.500e+01, 9.937e+03, ..., 0.000e+00, 1.000e+00,
        0.000e+00]])

In [7]:
prices = train['SalePrice']
prices = np.array(prices)
prices

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

X_train, X_test, y_train, y_test = train_test_split(inputs, prices,test_size=0.33, random_state=42)


In [9]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [127000 260000 110000 117000 270000  52000 230000 148500  52000 165150]
First 10 Actual labels: [154500, 325000, 115000, 159000, 315500, 75500, 311500, 146000, 84500, 135500]


In [16]:


error = 0
for x in range(len(predictions)):
    error += abs(predictions[x] - y_test[x])/y_test[x]
    
print(error/len(predictions))

0.22762730576956902


In [22]:
classifier = LogisticRegression(2.0, penalty='l2')

# for c in range(1, 10):
#     for penalty in ['l1', 'l2']:
#         classifier = LogisticRegression(c=1, penalty=penalty)
# classifier

TypeError: __init__() got multiple values for argument 'penalty'

In [66]:
from sklearn.model_selection import RepeatedKFold 
kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None) 

for train_index, test_index in kf.split(inputs):
      print("Train:", train_index, "Validation:",test_index)
      X_train, X_test = inputs[train_index], inputs[test_index] 
      y_train, y_test = prices[train_index], prices[test_index]

Train: [   0    1    2 ... 1457 1458 1459] Validation: [   4    7   16   21   24   34   39   46   48   53   70   79   88   98
  100  105  106  129  132  139  143  161  165  176  183  184  185  188
  189  195  198  200  207  210  220  221  222  224  225  232  237  244
  246  251  266  270  271  274  278  286  288  289  291  293  294  296
  297  299  301  303  304  315  323  327  333  340  344  345  349  353
  354  360  365  366  369  375  379  388  397  400  404  410  414  422
  424  434  435  440  452  454  457  459  462  466  475  494  499  504
  510  514  521  524  526  530  532  534  540  542  546  555  567  568
  577  581  586  587  591  601  604  606  609  610  612  615  622  629
  636  637  646  656  658  659  660  661  664  667  677  683  684  687
  688  691  693  697  699  708  710  711  712  726  730  733  736  737
  738  742  746  753  754  761  763  787  788  789  792  795  814  816
  821  826  827  830  832  834  842  846  850  851  873  874  877  878
  879  881  882  889  

Train: [   0    1    2 ... 1456 1458 1459] Validation: [   4    6    7   10   11   12   16   17   20   24   28   29   32   38
   42   49   55   57   70   83   97  104  112  121  123  124  127  131
  134  135  136  137  150  152  153  157  162  164  173  178  180  185
  189  193  202  210  212  213  219  228  235  236  237  245  254  263
  267  282  286  292  308  309  314  315  340  354  364  368  376  380
  392  405  411  420  423  427  429  430  435  438  447  450  452  456
  460  461  466  467  471  482  486  487  490  497  501  505  517  524
  530  534  540  550  555  568  575  577  578  583  584  601  604  608
  611  620  621  629  636  642  645  650  652  657  662  666  678  687
  702  706  712  727  730  731  733  740  741  751  754  764  767  771
  773  775  777  779  783  784  791  794  795  800  802  809  810  811
  812  813  817  818  819  828  831  842  848  860  861  862  865  870
  874  889  902  916  930  936  937  939  940  942  958  962  964  984
  987  989  992  998 1

In [67]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [68]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [69]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8904109589041096
Testing Data Score: 0.010273972602739725


In [70]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=None)
# X is the feature set and y is the target
for train_index, test_index in skf.split(inputs,prices): 
    print("Train:", train_index, "Validation:",test_index) 
    X_train, X_test = inputs[train_index], inputs[test_index] 
    y_train, y_test = prices[train_index], prices[test_index]


C:\Users\Shetu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Train: [  23   71  120  130  131  134  149  159  162  177  179  180  184  191
  194  203  205  207  221  222  228  242  248  249  251  255  256  257
  258  259  263  266  271  272  275  277  279  281  282  286  288  293
  302  307  316  318  325  331  334  337  338  341  343  344  348  351
  354  356  358  360  361  364  366  368  372  374  376  380  385  387
  393  394  396  397  405  406  407  408  411  413  414  417  421  422
  423  424  434  437  439  441  443  444  446  449  450  451  452  453
  455  456  459  461  466  468  470  471  472  482  486  487  490  493
  497  498  499  500  502  504  505  506  508  509  512  513  516  518
  521  522  525  526  528  530  534  535  536  538  540  546  547  548
  550  554  555  556  557  558  559  562  563  565  567  571  573  574
  576  577  582  583  584  586  587  588  593  594  599  600  601  602
  603  605  606  607  609  611  613  620  623  627  629  632  633  634
  635  636  638  641  642  643  647  648  649  651  652  653  655  657

In [71]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [72]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [73]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8845872899926954
Testing Data Score: 0.01098901098901099


In [74]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [149000 318061 165000 152000 217000 132500 109008 229000 191000 328000]
First 10 Actual labels: [108000, 141000, 152000, 172500, 207500, 132000, 110000, 220000, 192000, 187500]


In [75]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(inputs)

for train_index, test_index in loo.split(inputs):
        print("train:", train_index, "validation:", test_index)
        X_train, X_test = inputs[train_index], inputs[test_index]
        y_train, y_test = prices[train_index], prices[test_index]

train: [   1    2    3 ... 1457 1458 1459] validation: [0]
train: [   0    2    3 ... 1457 1458 1459] validation: [1]
train: [   0    1    3 ... 1457 1458 1459] validation: [2]
train: [   0    1    2 ... 1457 1458 1459] validation: [3]
train: [   0    1    2 ... 1457 1458 1459] validation: [4]
train: [   0    1    2 ... 1457 1458 1459] validation: [5]
train: [   0    1    2 ... 1457 1458 1459] validation: [6]
train: [   0    1    2 ... 1457 1458 1459] validation: [7]
train: [   0    1    2 ... 1457 1458 1459] validation: [8]
train: [   0    1    2 ... 1457 1458 1459] validation: [9]
train: [   0    1    2 ... 1457 1458 1459] validation: [10]
train: [   0    1    2 ... 1457 1458 1459] validation: [11]
train: [   0    1    2 ... 1457 1458 1459] validation: [12]
train: [   0    1    2 ... 1457 1458 1459] validation: [13]
train: [   0    1    2 ... 1457 1458 1459] validation: [14]
train: [   0    1    2 ... 1457 1458 1459] validation: [15]
train: [   0    1    2 ... 1457 1458 1459] validat

train: [   0    1    2 ... 1457 1458 1459] validation: [167]
train: [   0    1    2 ... 1457 1458 1459] validation: [168]
train: [   0    1    2 ... 1457 1458 1459] validation: [169]
train: [   0    1    2 ... 1457 1458 1459] validation: [170]
train: [   0    1    2 ... 1457 1458 1459] validation: [171]
train: [   0    1    2 ... 1457 1458 1459] validation: [172]
train: [   0    1    2 ... 1457 1458 1459] validation: [173]
train: [   0    1    2 ... 1457 1458 1459] validation: [174]
train: [   0    1    2 ... 1457 1458 1459] validation: [175]
train: [   0    1    2 ... 1457 1458 1459] validation: [176]
train: [   0    1    2 ... 1457 1458 1459] validation: [177]
train: [   0    1    2 ... 1457 1458 1459] validation: [178]
train: [   0    1    2 ... 1457 1458 1459] validation: [179]
train: [   0    1    2 ... 1457 1458 1459] validation: [180]
train: [   0    1    2 ... 1457 1458 1459] validation: [181]
train: [   0    1    2 ... 1457 1458 1459] validation: [182]
train: [   0    1    2 .

train: [   0    1    2 ... 1457 1458 1459] validation: [321]
train: [   0    1    2 ... 1457 1458 1459] validation: [322]
train: [   0    1    2 ... 1457 1458 1459] validation: [323]
train: [   0    1    2 ... 1457 1458 1459] validation: [324]
train: [   0    1    2 ... 1457 1458 1459] validation: [325]
train: [   0    1    2 ... 1457 1458 1459] validation: [326]
train: [   0    1    2 ... 1457 1458 1459] validation: [327]
train: [   0    1    2 ... 1457 1458 1459] validation: [328]
train: [   0    1    2 ... 1457 1458 1459] validation: [329]
train: [   0    1    2 ... 1457 1458 1459] validation: [330]
train: [   0    1    2 ... 1457 1458 1459] validation: [331]
train: [   0    1    2 ... 1457 1458 1459] validation: [332]
train: [   0    1    2 ... 1457 1458 1459] validation: [333]
train: [   0    1    2 ... 1457 1458 1459] validation: [334]
train: [   0    1    2 ... 1457 1458 1459] validation: [335]
train: [   0    1    2 ... 1457 1458 1459] validation: [336]
train: [   0    1    2 .

train: [   0    1    2 ... 1457 1458 1459] validation: [487]
train: [   0    1    2 ... 1457 1458 1459] validation: [488]
train: [   0    1    2 ... 1457 1458 1459] validation: [489]
train: [   0    1    2 ... 1457 1458 1459] validation: [490]
train: [   0    1    2 ... 1457 1458 1459] validation: [491]
train: [   0    1    2 ... 1457 1458 1459] validation: [492]
train: [   0    1    2 ... 1457 1458 1459] validation: [493]
train: [   0    1    2 ... 1457 1458 1459] validation: [494]
train: [   0    1    2 ... 1457 1458 1459] validation: [495]
train: [   0    1    2 ... 1457 1458 1459] validation: [496]
train: [   0    1    2 ... 1457 1458 1459] validation: [497]
train: [   0    1    2 ... 1457 1458 1459] validation: [498]
train: [   0    1    2 ... 1457 1458 1459] validation: [499]
train: [   0    1    2 ... 1457 1458 1459] validation: [500]
train: [   0    1    2 ... 1457 1458 1459] validation: [501]
train: [   0    1    2 ... 1457 1458 1459] validation: [502]
train: [   0    1    2 .

train: [   0    1    2 ... 1457 1458 1459] validation: [656]
train: [   0    1    2 ... 1457 1458 1459] validation: [657]
train: [   0    1    2 ... 1457 1458 1459] validation: [658]
train: [   0    1    2 ... 1457 1458 1459] validation: [659]
train: [   0    1    2 ... 1457 1458 1459] validation: [660]
train: [   0    1    2 ... 1457 1458 1459] validation: [661]
train: [   0    1    2 ... 1457 1458 1459] validation: [662]
train: [   0    1    2 ... 1457 1458 1459] validation: [663]
train: [   0    1    2 ... 1457 1458 1459] validation: [664]
train: [   0    1    2 ... 1457 1458 1459] validation: [665]
train: [   0    1    2 ... 1457 1458 1459] validation: [666]
train: [   0    1    2 ... 1457 1458 1459] validation: [667]
train: [   0    1    2 ... 1457 1458 1459] validation: [668]
train: [   0    1    2 ... 1457 1458 1459] validation: [669]
train: [   0    1    2 ... 1457 1458 1459] validation: [670]
train: [   0    1    2 ... 1457 1458 1459] validation: [671]
train: [   0    1    2 .

train: [   0    1    2 ... 1457 1458 1459] validation: [817]
train: [   0    1    2 ... 1457 1458 1459] validation: [818]
train: [   0    1    2 ... 1457 1458 1459] validation: [819]
train: [   0    1    2 ... 1457 1458 1459] validation: [820]
train: [   0    1    2 ... 1457 1458 1459] validation: [821]
train: [   0    1    2 ... 1457 1458 1459] validation: [822]
train: [   0    1    2 ... 1457 1458 1459] validation: [823]
train: [   0    1    2 ... 1457 1458 1459] validation: [824]
train: [   0    1    2 ... 1457 1458 1459] validation: [825]
train: [   0    1    2 ... 1457 1458 1459] validation: [826]
train: [   0    1    2 ... 1457 1458 1459] validation: [827]
train: [   0    1    2 ... 1457 1458 1459] validation: [828]
train: [   0    1    2 ... 1457 1458 1459] validation: [829]
train: [   0    1    2 ... 1457 1458 1459] validation: [830]
train: [   0    1    2 ... 1457 1458 1459] validation: [831]
train: [   0    1    2 ... 1457 1458 1459] validation: [832]
train: [   0    1    2 .

train: [   0    1    2 ... 1457 1458 1459] validation: [981]
train: [   0    1    2 ... 1457 1458 1459] validation: [982]
train: [   0    1    2 ... 1457 1458 1459] validation: [983]
train: [   0    1    2 ... 1457 1458 1459] validation: [984]
train: [   0    1    2 ... 1457 1458 1459] validation: [985]
train: [   0    1    2 ... 1457 1458 1459] validation: [986]
train: [   0    1    2 ... 1457 1458 1459] validation: [987]
train: [   0    1    2 ... 1457 1458 1459] validation: [988]
train: [   0    1    2 ... 1457 1458 1459] validation: [989]
train: [   0    1    2 ... 1457 1458 1459] validation: [990]
train: [   0    1    2 ... 1457 1458 1459] validation: [991]
train: [   0    1    2 ... 1457 1458 1459] validation: [992]
train: [   0    1    2 ... 1457 1458 1459] validation: [993]
train: [   0    1    2 ... 1457 1458 1459] validation: [994]
train: [   0    1    2 ... 1457 1458 1459] validation: [995]
train: [   0    1    2 ... 1457 1458 1459] validation: [996]
train: [   0    1    2 .

train: [   0    1    2 ... 1457 1458 1459] validation: [1153]
train: [   0    1    2 ... 1457 1458 1459] validation: [1154]
train: [   0    1    2 ... 1457 1458 1459] validation: [1155]
train: [   0    1    2 ... 1457 1458 1459] validation: [1156]
train: [   0    1    2 ... 1457 1458 1459] validation: [1157]
train: [   0    1    2 ... 1457 1458 1459] validation: [1158]
train: [   0    1    2 ... 1457 1458 1459] validation: [1159]
train: [   0    1    2 ... 1457 1458 1459] validation: [1160]
train: [   0    1    2 ... 1457 1458 1459] validation: [1161]
train: [   0    1    2 ... 1457 1458 1459] validation: [1162]
train: [   0    1    2 ... 1457 1458 1459] validation: [1163]
train: [   0    1    2 ... 1457 1458 1459] validation: [1164]
train: [   0    1    2 ... 1457 1458 1459] validation: [1165]
train: [   0    1    2 ... 1457 1458 1459] validation: [1166]
train: [   0    1    2 ... 1457 1458 1459] validation: [1167]
train: [   0    1    2 ... 1457 1458 1459] validation: [1168]
train: [

train: [   0    1    2 ... 1457 1458 1459] validation: [1288]
train: [   0    1    2 ... 1457 1458 1459] validation: [1289]
train: [   0    1    2 ... 1457 1458 1459] validation: [1290]
train: [   0    1    2 ... 1457 1458 1459] validation: [1291]
train: [   0    1    2 ... 1457 1458 1459] validation: [1292]
train: [   0    1    2 ... 1457 1458 1459] validation: [1293]
train: [   0    1    2 ... 1457 1458 1459] validation: [1294]
train: [   0    1    2 ... 1457 1458 1459] validation: [1295]
train: [   0    1    2 ... 1457 1458 1459] validation: [1296]
train: [   0    1    2 ... 1457 1458 1459] validation: [1297]
train: [   0    1    2 ... 1457 1458 1459] validation: [1298]
train: [   0    1    2 ... 1457 1458 1459] validation: [1299]
train: [   0    1    2 ... 1457 1458 1459] validation: [1300]
train: [   0    1    2 ... 1457 1458 1459] validation: [1301]
train: [   0    1    2 ... 1457 1458 1459] validation: [1302]
train: [   0    1    2 ... 1457 1458 1459] validation: [1303]
train: [

train: [   0    1    2 ... 1457 1458 1459] validation: [1438]
train: [   0    1    2 ... 1457 1458 1459] validation: [1439]
train: [   0    1    2 ... 1457 1458 1459] validation: [1440]
train: [   0    1    2 ... 1457 1458 1459] validation: [1441]
train: [   0    1    2 ... 1457 1458 1459] validation: [1442]
train: [   0    1    2 ... 1457 1458 1459] validation: [1443]
train: [   0    1    2 ... 1457 1458 1459] validation: [1444]
train: [   0    1    2 ... 1457 1458 1459] validation: [1445]
train: [   0    1    2 ... 1457 1458 1459] validation: [1446]
train: [   0    1    2 ... 1457 1458 1459] validation: [1447]
train: [   0    1    2 ... 1457 1458 1459] validation: [1448]
train: [   0    1    2 ... 1457 1458 1459] validation: [1449]
train: [   0    1    2 ... 1457 1458 1459] validation: [1450]
train: [   0    1    2 ... 1457 1458 1459] validation: [1451]
train: [   0    1    2 ... 1457 1458 1459] validation: [1452]
train: [   0    1    2 ... 1457 1458 1459] validation: [1453]
train: [

In [76]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [77]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [78]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8444139821795751
Testing Data Score: 0.0


In [79]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [138500]
First 10 Actual labels: [147500]


In [1]:
len(predictions)

NameError: name 'predictions' is not defined

In [8]:
#from sklearn.linear_model import LogisticRegressionCV
#classifier = LogisticRegressionCV(cv=5, random_state=0).fit(X_train, y_train)


C:\Users\Shetu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [33]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9222903885480572
Testing Data Score: 0.002074688796680498


In [37]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [38]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [39]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


C:\Users\Shetu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


[CV] C=1, gamma=0.0001 ...............................................
[CV] .... C=1, gamma=0.0001, score=0.018211920529801324, total=   1.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV] .... C=1, gamma=0.0001, score=0.012552301255230125, total=   3.7s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    9.3s remaining:    0.0s


[CV] .... C=1, gamma=0.0001, score=0.014814814814814815, total=   4.4s
[CV] C=1, gamma=0.001 ................................................
[CV] ..... C=1, gamma=0.001, score=0.018211920529801324, total=   1.4s
[CV] C=1, gamma=0.001 ................................................
[CV] ..... C=1, gamma=0.001, score=0.012552301255230125, total=   4.0s
[CV] C=1, gamma=0.001 ................................................
[CV] ..... C=1, gamma=0.001, score=0.014814814814814815, total=   4.2s
[CV] C=1, gamma=0.01 .................................................
[CV] ...... C=1, gamma=0.01, score=0.018211920529801324, total=   1.0s
[CV] C=1, gamma=0.01 .................................................
[CV] ...... C=1, gamma=0.01, score=0.012552301255230125, total=   3.7s
[CV] C=1, gamma=0.01 .................................................
[CV] ...... C=1, gamma=0.01, score=0.014814814814814815, total=   4.3s
[CV] C=5, gamma=0.0001 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [40]:
print(grid.best_params_)

{'C': 1, 'gamma': 0.0001}


In [41]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9222903885480572
Testing Data Score: 0.002074688796680498


In [45]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)
for train_index, test_index in tscv.split(inputs):
     print("Train:", train_index, "Validation:", test_index)
     inputs_train, inputs_test = inputs[train_index], inputs[test_index]
     prices_train, prices_test = prices[train_index], prices[test_index]


Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

In [46]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [49]:
classifier.fit(inputs_train, prices_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
print(f"Training Data Score: {classifier.score(inputs_train, prices_train)}")
print(f"Testing Data Score: {classifier.score(inputs_test, prices_test)}")

Training Data Score: 0.9059360730593607
Testing Data Score: 0.005479452054794521
